In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="destination", description="여행지 출력"),
    ResponseSchema(name="duration", description="여행 기간을 n박 n일 단위로 출력"),
    ResponseSchema(name="budget", description="여행 경비 출력"),
    ResponseSchema(name="rating", description="여행 만족도 점수 1점에서 5점 사이의 숫자로 출력"),
    ResponseSchema(name="activities", description="여행 중 하는 활동들을 리스트 형태로 출력")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
# 프롬프트 템플릿
template = """
다음 여행기에서 요청된 5개의 필드 정보를 추출하세요.
요청 스키마의 필드 순서대로(destination, duration, budget, rating, activities) 추출하세요.
여행기: {travelogue}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 테스트 리뷰 데이터
travelogue = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 
정말 만족스러운 여행이었습니다. 
5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"travelogue": travelogue})

# 결과 출력 (Pretty Print)
print("===== 여행 분석 결과 =====")
pprint(output)

===== 여행 분석 결과 =====
{'activities': '[해운대에서 바다구경, 자갈치시장에서 회 먹기, 감천문화마을 구경]',
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4'}
